<a href="https://colab.research.google.com/github/2303A52054/HPC/blob/main/ass3_2303A52054.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import numba
from numba import njit, prange
import time

# Task 1: Serial Python function
@njit
def vector_saxpy_serial(alpha, A, B, C):
    """Serial implementation: C[i] = alpha * A[i] + B[i]"""
    n = len(A)
    for i in range(n):
        C[i] = alpha * A[i] + B[i]
    return C

# Task 2: Parallel version using prange
@njit(parallel=True)
def vector_saxpy_parallel(alpha, A, B, C):
    """Parallel implementation using prange"""
    n = len(A)
    for i in prange(n):
        C[i] = alpha * A[i] + B[i]
    return C

# Task 3 & 4: Measure runtime and compare performance
def benchmark_vector_operations():
    """Measure runtime for increasing vector sizes"""
    sizes = [1_000_000, 5_000_000, 10_000_000, 50_000_000, 100_000_000]
    alpha = 2.5

    print("Vector Size\tSerial Time(s)\tParallel Time(s)\tSpeedup")
    print("-" * 65)

    for size in sizes:
        A = np.random.rand(size).astype(np.float64)
        B = np.random.rand(size).astype(np.float64)
        C_serial = np.zeros(size, dtype=np.float64)
        C_parallel = np.zeros(size, dtype=np.float64)

        # Warm-up
        vector_saxpy_serial(alpha, A[:1000], B[:1000], C_serial[:1000])
        vector_saxpy_parallel(alpha, A[:1000], B[:1000], C_parallel[:1000])

        # Serial timing
        start = time.perf_counter()
        vector_saxpy_serial(alpha, A, B, C_serial)
        serial_time = time.perf_counter() - start

        # Parallel timing
        start = time.perf_counter()
        vector_saxpy_parallel(alpha, A, B, C_parallel)
        parallel_time = time.perf_counter() - start

        speedup = serial_time / parallel_time

        print(f"{size:>11,}\t{serial_time:.4f}\t\t{parallel_time:.4f}\t\t\t{speedup:.2f}x")

if __name__ == "__main__":
    benchmark_vector_operations()

Vector Size	Serial Time(s)	Parallel Time(s)	Speedup
-----------------------------------------------------------------
  1,000,000	0.0029		0.0065			0.45x
  5,000,000	0.0322		0.0282			1.14x
 10,000,000	0.0697		0.0748			0.93x
 50,000,000	0.4359		0.3648			1.19x
100,000,000	0.6693		0.5516			1.21x


In [3]:
import numpy as np
import numba
from numba import njit, prange
import time

# Task 1: Serial matrix multiplication
@njit
def matrix_multiply_serial(A, B, C):
    """Serial matrix multiplication"""
    n = A.shape[0]
    m = B.shape[1]
    k = A.shape[1]

    for i in range(n):
        for j in range(m):
            temp = 0.0
            for p in range(k):
                temp += A[i, p] * B[p, j]
            C[i, j] = temp
    return C

# Task 2: Parallelize outer loop
@njit(parallel=True)
def matrix_multiply_parallel_outer(A, B, C):
    """Parallel matrix multiplication - outer loop parallelized"""
    n = A.shape[0]
    m = B.shape[1]
    k = A.shape[1]

    for i in prange(n):
        for j in range(m):
            temp = 0.0
            for p in range(k):
                temp += A[i, p] * B[p, j]
            C[i, j] = temp
    return C

# Task 3: Parallelize using collapsed loops logic
@njit(parallel=True)
def matrix_multiply_parallel_collapsed(A, B, C):
    """Parallel matrix multiplication - collapsed loop approach"""
    n = A.shape[0]
    m = B.shape[1]
    k = A.shape[1]

    for i in prange(n * m):
        row = i // m
        col = i % m
        temp = 0.0
        for p in range(k):
            temp += A[row, p] * B[p, col]
        C[row, col] = temp
    return C

# Task 4: Analyze cache behavior and performance
def benchmark_matrix_multiplication():
    """Compare serial vs parallel performance"""
    sizes = [128, 256, 512, 1024]

    print("Matrix Size\tSerial(s)\tParallel Outer(s)\tCollapsed(s)\tSpeedup(Outer)\tSpeedup(Collapsed)")
    print("-" * 100)

    for size in sizes:
        A = np.random.rand(size, size).astype(np.float64)
        B = np.random.rand(size, size).astype(np.float64)
        C_serial = np.zeros((size, size), dtype=np.float64)
        C_parallel_outer = np.zeros((size, size), dtype=np.float64)
        C_parallel_collapsed = np.zeros((size, size), dtype=np.float64)

        # Warm-up
        matrix_multiply_serial(A[:10, :10], B[:10, :10], C_serial[:10, :10])
        matrix_multiply_parallel_outer(A[:10, :10], B[:10, :10], C_parallel_outer[:10, :10])
        matrix_multiply_parallel_collapsed(A[:10, :10], B[:10, :10], C_parallel_collapsed[:10, :10])

        # Serial timing
        start = time.perf_counter()
        matrix_multiply_serial(A, B, C_serial)
        serial_time = time.perf_counter() - start

        # Parallel outer timing
        start = time.perf_counter()
        matrix_multiply_parallel_outer(A, B, C_parallel_outer)
        parallel_outer_time = time.perf_counter() - start

        # Parallel collapsed timing
        start = time.perf_counter()
        matrix_multiply_parallel_collapsed(A, B, C_parallel_collapsed)
        parallel_collapsed_time = time.perf_counter() - start

        speedup_outer = serial_time / parallel_outer_time
        speedup_collapsed = serial_time / parallel_collapsed_time

        print(f"{size}x{size}\t\t{serial_time:.4f}\t\t{parallel_outer_time:.4f}\t\t\t{parallel_collapsed_time:.4f}\t\t{speedup_outer:.2f}x\t\t{speedup_collapsed:.2f}x")

if __name__ == "__main__":
    benchmark_matrix_multiplication()

Matrix Size	Serial(s)	Parallel Outer(s)	Collapsed(s)	Speedup(Outer)	Speedup(Collapsed)
----------------------------------------------------------------------------------------------------
128x128		0.2189		1.0196			0.8370		0.21x		0.26x
256x256		0.0238		0.0167			0.0171		1.42x		1.39x
512x512		0.3745		0.3034			0.3421		1.23x		1.09x
1024x1024		6.0037		3.7350			3.7350		1.61x		1.61x


In [ ]:
import numpy as np
import numba
from numba import njit, prange
import time

# Task 1: Simulate variable workload per iteration
@njit
def process_image_serial(image_sizes, results):
    """Serial processing with variable workload"""
    n = len(image_sizes)
    for i in range(n):
        # Simulate variable computation based on image size
        workload = image_sizes[i]
        computation = 0.0
        for j in range(workload):
            computation += np.sqrt(j + 1) * np.sin(j * 0.1)
        results[i] = computation
    return results

# Task 2: Parallelize using prange
@njit(parallel=True)
def process_image_parallel(image_sizes, results):
    """Parallel processing with variable workload"""
    n = len(image_sizes)
    for i in prange(n):
        # Simulate variable computation based on image size
        workload = image_sizes[i]
        computation = 0.0
        for j in range(workload):
            computation += np.sqrt(j + 1) * np.sin(j * 0.1)
        results[i] = computation
    return results

# Task 3: Observe execution time variation
def benchmark_load_imbalance():
    """Analyze load imbalance effects"""
    n_images = 1000

    # Create different workload distributions
    print("Workload Distribution\tSerial Time(s)\tParallel Time(s)\tSpeedup\t\tEfficiency")
    print("-" * 90)

    # Uniform workload
    uniform_sizes = np.full(n_images, 10000, dtype=np.int32)
    results_serial = np.zeros(n_images, dtype=np.float64)
    results_parallel = np.zeros(n_images, dtype=np.float64)

    start = time.perf_counter()
    process_image_serial(uniform_sizes, results_serial)
    serial_time = time.perf_counter() - start

    start = time.perf_counter()
    process_image_parallel(uniform_sizes, results_parallel)
    parallel_time = time.perf_counter() - start

    speedup = serial_time / parallel_time
    efficiency = speedup / numba.config.NUMBA_NUM_THREADS * 100
    print(f"Uniform\t\t\t{serial_time:.4f}\t\t{parallel_time:.4f}\t\t{speedup:.2f}x\t\t{efficiency:.1f}%")

    # Imbalanced workload (ascending)
    imbalanced_sizes = np.linspace(1000, 20000, n_images, dtype=np.int32)
    results_serial = np.zeros(n_images, dtype=np.float64)
    results_parallel = np.zeros(n_images, dtype=np.float64)

    start = time.perf_counter()
    process_image_serial(imbalanced_sizes, results_serial)
    serial_time = time.perf_counter() - start

    start = time.perf_counter()
    process_image_parallel(imbalanced_sizes, results_parallel)
    parallel_time = time.perf_counter() - start

    speedup = serial_time / parallel_time
    efficiency = speedup / numba.config.NUMBA_NUM_THREADS * 100
    print(f"Imbalanced (Ascending)\t{serial_time:.4f}\t\t{parallel_time:.4f}\t\t{speedup:.2f}x\t\t{efficiency:.1f}%")

    # Random workload
    np.random.seed(42)
    random_sizes = np.random.randint(1000, 20000, n_images, dtype=np.int32)
    results_serial = np.zeros(n_images, dtype=np.float64)
    results_parallel = np.zeros(n_images, dtype=np.float64)

    start = time.perf_counter()
    process_image_serial(random_sizes, results_serial)
    serial_time = time.perf_counter() - start

    start = time.perf_counter()
    process_image_parallel(random_sizes, results_parallel)
    parallel_time = time.perf_counter() - start

    speedup = serial_time / parallel_time
    efficiency = speedup / numba.config.NUMBA_NUM_THREADS * 100
    print(f"Random\t\t\t{serial_time:.4f}\t\t{parallel_time:.4f}\t\t{speedup:.2f}x\t\t{efficiency:.1f}%")

# Task 4: Discussion on limitations
def print_scheduling_limitations():
    """Discuss Python/Numba scheduling limitations"""
    print("\n" + "="*80)
    print("SCHEDULING LIMITATIONS IN PYTHON/NUMBA")
    print("="*80)
    print("\n1. Static Scheduling:")
    print("   - Numba's prange uses static scheduling by default")
    print("   - Work is divided equally among threads at compile time")
    print("   - No runtime load balancing unlike OpenMP's dynamic/guided scheduling")

    print("\n2. Load Imbalance Impact:")
    print("   - Threads with lighter work finish early and remain idle")
    print("   - Thread with heaviest work determines total execution time")
    print("   - Efficiency degrades with increasing workload variance")

    print("\n3. OpenMP Comparison:")
    print("   - OpenMP offers: schedule(static), schedule(dynamic), schedule(guided)")
    print("   - Numba lacks equivalent scheduling control")
    print("   - No work-stealing or dynamic chunk assignment in Numba")

    print("\n4. Workarounds:")
    print("   - Pre-sort tasks by workload (decreasing order)")
    print("   - Use task parallelism instead of data parallelism")
    print("   - Consider Dask or Ray for better load balancing")
    print("="*80 + "\n")

if __name__ == "__main__":
    benchmark_load_imbalance()
    print_scheduling_limitations()

In [2]:
import numpy as np
import numba
from numba import njit, prange
import time

# Task 1 & 2: Serial versions
@njit
def compute_sum_serial(data):
    """Serial sum computation"""
    total = 0.0
    for i in range(len(data)):
        total += data[i]
    return total

@njit
def compute_max_serial(data):
    """Serial maximum computation"""
    max_val = data[0]
    for i in range(1, len(data)):
        if data[i] > max_val:
            max_val = data[i]
    return max_val

# Task 2: Parallel versions
@njit(parallel=True)
def compute_sum_parallel(data):
    """Parallel sum using reduction"""
    total = 0.0
    for i in prange(len(data)):
        total += data[i]
    return total

@njit(parallel=True)
def compute_max_parallel(data):
    """Parallel maximum using reduction"""
    n = len(data)
    max_val = data[0]
    for i in prange(n):
        if data[i] > max_val:
            max_val = data[i]
    return max_val

# Alternative: Manual reduction with local maxima
@njit(parallel=True)
def compute_sum_parallel_manual(data):
    """Manual parallel sum with thread-local accumulation"""
    n = len(data)
    num_threads = numba.get_num_threads()
    chunk_size = (n + num_threads - 1) // num_threads
    partial_sums = np.zeros(num_threads, dtype=np.float64)

    for i in prange(n):
        thread_id = i // chunk_size
        if thread_id < num_threads:
            partial_sums[thread_id] += data[i]

    total = 0.0
    for i in range(num_threads):
        total += partial_sums[i]
    return total

# Task 3: Verify correctness and performance
def benchmark_reductions():
    """Test correctness and measure performance"""
    sizes = [10_000_000, 50_000_000, 100_000_000, 500_000_000]

    print("="*90)
    print("PARALLEL REDUCTION OPERATIONS - SUM")
    print("="*90)
    print("Data Size\tSerial Time(s)\tParallel Time(s)\tSpeedup\t\tCorrect")
    print("-" * 90)

    for size in sizes:
        data = np.random.rand(size).astype(np.float64)

        # Warm-up
        compute_sum_serial(data[:1000])
        compute_sum_parallel(data[:1000])

        # Serial sum
        start = time.perf_counter()
        sum_serial = compute_sum_serial(data)
        serial_time = time.perf_counter() - start

        # Parallel sum
        start = time.perf_counter()
        sum_parallel = compute_sum_parallel(data)
        parallel_time = time.perf_counter() - start

        speedup = serial_time / parallel_time
        correct = np.isclose(sum_serial, sum_parallel, rtol=1e-5)

        print(f"{size:>11,}\t{serial_time:.4f}\t\t{parallel_time:.4f}\t\t\t{speedup:.2f}x\t\t{correct}")

    print("\n" + "="*90)
    print("PARALLEL REDUCTION OPERATIONS - MAXIMUM")
    print("="*90)
    print("Data Size\tSerial Time(s)\tParallel Time(s)\tSpeedup\t\tCorrect")
    print("-" * 90)

    for size in sizes:
        data = np.random.rand(size).astype(np.float64)

        # Warm-up
        compute_max_serial(data[:1000])
        compute_max_parallel(data[:1000])

        # Serial max
        start = time.perf_counter()
        max_serial = compute_max_serial(data)
        serial_time = time.perf_counter() - start

        # Parallel max
        start = time.perf_counter()
        max_parallel = compute_max_parallel(data)
        parallel_time = time.perf_counter() - start

        speedup = serial_time / parallel_time
        correct = np.isclose(max_serial, max_parallel)

        print(f"{size:>11,}\t{serial_time:.4f}\t\t{parallel_time:.4f}\t\t\t{speedup:.2f}x\t\t{correct}")

if __name__ == "__main__":
    benchmark_reductions()

PARALLEL REDUCTION OPERATIONS - SUM
Data Size	Serial Time(s)	Parallel Time(s)	Speedup		Correct
------------------------------------------------------------------------------------------
 10,000,000	0.0175		0.0115			1.52x		True
 50,000,000	0.0837		0.0463			1.81x		True
100,000,000	0.1725		0.1069			1.61x		True
500,000,000	0.8376		0.5052			1.66x		True

PARALLEL REDUCTION OPERATIONS - MAXIMUM
Data Size	Serial Time(s)	Parallel Time(s)	Speedup		Correct
------------------------------------------------------------------------------------------
 10,000,000	0.0175		0.0001			201.97x		False
 50,000,000	0.0954		0.0001			1000.96x		False
100,000,000	0.1891		0.0001			2153.12x		False
500,000,000	0.8710		0.0001			12723.37x		False


In [1]:
import numpy as np
import numba
from numba import njit, prange
import time

# Task 1: Serial Monte Carlo simulation
@njit
def monte_carlo_pi_serial(n_samples):
    """Serial Monte Carlo estimation of pi"""
    count_inside = 0

    # Use Numba-compatible random number generation
    np.random.seed(42)

    for i in range(n_samples):
        x = np.random.random()
        y = np.random.random()

        if x*x + y*y <= 1.0:
            count_inside += 1

    pi_estimate = 4.0 * count_inside / n_samples
    return pi_estimate

# Task 2: Parallel version using prange
@njit(parallel=True)
def monte_carlo_pi_parallel(n_samples):
    """Parallel Monte Carlo estimation of pi using prange"""
    count_inside = 0

    # Set seed for reproducibility (each thread will have different subsequence)
    np.random.seed(42)

    for i in prange(n_samples):
        x = np.random.random()
        y = np.random.random()

        if x*x + y*y <= 1.0:
            count_inside += 1

    pi_estimate = 4.0 * count_inside / n_samples
    return pi_estimate

# Alternative: Thread-safe version with per-thread RNG
@njit(parallel=True)
def monte_carlo_pi_parallel_safe(n_samples):
    """Parallel Monte Carlo with thread-local counting"""
    num_threads = numba.get_num_threads()
    counts = np.zeros(num_threads, dtype=np.int64)

    for i in prange(n_samples):
        # Generate random numbers
        state = i + 12345  # Simple seed based on iteration
        x = (state * 1103515245 + 12345) % 2147483648 / 2147483648.0
        y = ((state + 1) * 1103515245 + 12345) % 2147483648 / 2147483648.0

        if x*x + y*y <= 1.0:
            thread_id = numba.get_thread_id()
            counts[thread_id] += 1

    total_inside = np.sum(counts)
    pi_estimate = 4.0 * total_inside / n_samples
    return pi_estimate

# Task 3 & 4: Experiments and timing
def benchmark_monte_carlo():
    """Measure execution time and speedup for different sample sizes"""
    sample_sizes = [50_000_000, 100_000_000, 500_000_000, 1_000_000_000]

    print("="*100)
    print("MONTE CARLO SIMULATION FOR π ESTIMATION")
    print("="*100)
    print("Samples\t\t\tSerial Time(s)\tParallel Time(s)\tSpeedup\t\tPi Estimate\t\tError")
    print("-" * 100)

    for n_samples in sample_sizes:
        # Warm-up
        monte_carlo_pi_serial(1000)
        monte_carlo_pi_parallel(1000)

        # Serial timing
        start = time.perf_counter()
        pi_serial = monte_carlo_pi_serial(n_samples)
        serial_time = time.perf_counter() - start

        # Parallel timing
        start = time.perf_counter()
        pi_parallel = monte_carlo_pi_parallel(n_samples)
        parallel_time = time.perf_counter() - start

        speedup = serial_time / parallel_time
        error = abs(pi_parallel - np.pi)

        print(f"{n_samples:>13,}\t{serial_time:>8.4f}\t{parallel_time:>10.4f}\t\t{speedup:>6.2f}x\t\t{pi_parallel:.6f}\t\t{error:.6f}")

# Task 5: Discussion on race conditions and reduction
def discuss_race_conditions():
    """Discuss race conditions and reduction handling"""
    print("\n" + "="*100)
    print("RACE CONDITIONS AND REDUCTION HANDLING")
    print("="*100)

    print("\n1. RACE CONDITION IN NAIVE PARALLEL IMPLEMENTATION:")
    print("   - Multiple threads increment 'count_inside' simultaneously")
    print("   - Without synchronization: lost updates, incorrect results")
    print("   - Example: Thread A reads count=100, Thread B reads count=100")
    print("             Both increment to 101, write back → one update lost!")

    print("\n2. NUMBA'S AUTOMATIC REDUCTION:")
    print("   - Numba recognizes reduction patterns (+=, min, max)")
    print("   - Implements thread-local accumulators automatically")
    print("   - Combines results at the end with proper synchronization")
    print("   - Similar to OpenMP's 'reduction(+:count_inside)'")

    print("\n3. REDUCTION MECHANISM:")
    print("   Step 1: Each thread maintains private counter")
    print("   Step 2: Threads count points independently (no contention)")
    print("   Step 3: Private counters combined with atomic operations")
    print("   Step 4: Final sum computed safely")

    print("\n4. RANDOM NUMBER GENERATION CONSIDERATIONS:")
    print("   - np.random.random() in parallel loops: each thread gets subsequence")
    print("   - Potential correlation between threads")
    print("   - For production: use per-thread RNG with different seeds")
    print("   - Alternative: XORShift, PCG, or other parallel-friendly PRNGs")

    print("\n5. PERFORMANCE FACTORS:")
    print("   - Reduction overhead: minimal for large n_samples")
    print("   - Memory bandwidth: each thread reads/writes independently")
    print("   - Cache effects: good locality within each thread")
    print("   - Scalability: near-linear up to memory bandwidth limit")

    print("\n6. VERIFICATION:")
    n_samples = 100_000_000
    pi_estimate = monte_carlo_pi_parallel(n_samples)
    error = abs(pi_estimate - np.pi)
    print(f"   π estimate with {n_samples:,} samples: {pi_estimate:.8f}")
    print(f"   Actual π value: {np.pi:.8f}")
    print(f"   Absolute error: {error:.8f}")
    print(f"   Relative error: {error/np.pi * 100:.4f}%")

    print("="*100 + "\n")

if __name__ == "__main__":
    benchmark_monte_carlo()
    discuss_race_conditions()

MONTE CARLO SIMULATION FOR π ESTIMATION
Samples			Serial Time(s)	Parallel Time(s)	Speedup		Pi Estimate		Error
----------------------------------------------------------------------------------------------------
   50,000,000	  0.6816	    0.6036		  1.13x		3.141988		0.000395
  100,000,000	  1.2716	    1.2363		  1.03x		3.141567		0.000026
  500,000,000	  7.6370	    6.1137		  1.25x		3.141622		0.000029
1,000,000,000	 14.7445	   13.2960		  1.11x		3.141542		0.000051

RACE CONDITIONS AND REDUCTION HANDLING

1. RACE CONDITION IN NAIVE PARALLEL IMPLEMENTATION:
   - Multiple threads increment 'count_inside' simultaneously
   - Without synchronization: lost updates, incorrect results
   - Example: Thread A reads count=100, Thread B reads count=100
             Both increment to 101, write back → one update lost!

2. NUMBA'S AUTOMATIC REDUCTION:
   - Numba recognizes reduction patterns (+=, min, max)
   - Implements thread-local accumulators automatically
   - Combines results at the end with proper